# SQL (SQLAlchemy)

> [Structured Query Language (SQL)](https://en.wikipedia.org/wiki/SQL) is a domain-specific language used in programming for managing data held in a relational database management system (RDBMS) or for stream processing in a relational data stream management system (RDSMS). It is particularly useful in handling structured data, i.e., data incorporating relations among entities and variables.

> [SQLAlchemy](https://github.com/sqlalchemy/sqlalchemy) is an open-source `SQL` toolkit and Object-Relational Mapping (ORM) library for the Python programming language, distributed under the MIT license.

In this notebook, we will describe the `SQLChatMessageHistory` class, which can store chat history in any database supported by `SQLAlchemy`.

To use it with databases other than `SQLite`, you need to install the corresponding database driver.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Usage

To use storage, you only need to provide the following two things:

1. `session_id` - A unique identifier for the session, such as a username, email, or chat ID.

2. `connection` - A string specifying the database connection. This string is passed to SQLAlchemy's create_engine function.


In [1]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)

In [5]:
# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "My name is Abby. I am a AI Developer. Glad to see you!"
)
# AI 메시지를 추가합니다.
chat_message_history.add_ai_message("Hello Abby, What can I help you!")

In [6]:
chat_message_history.messages

[HumanMessage(content='My name is Abby. I am a AI Developer. Glad to see you!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Abby, What can I help you!', additional_kwargs={}, response_metadata={})]

In [7]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [8]:
prompt = ChatPromptTemplate.from_messages(
    [
        
        ("system", "You are a helpful assistant."),
        
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),  # 질문
    ]
)

# chain 을 생성합니다.
chain = prompt | ChatOpenAI(model_name="gpt-4o-mini") | StrOutputParser()

In [9]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db",
    )

In [10]:
from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [11]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # Set the function to retrieve chat history.
    input_messages_key="question",  # Set the key for input messages to "question".
    history_messages_key="chat_history",  # Set the key for chat history messages to "history".
    history_factory_config=config_fields,  # Set the parameters to refer to when retrieving chat history.
)

In [12]:
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}

In [15]:
chain_with_history.invoke({"question": "Hi my name is Abby"}, config)

"Hi Abby! It's nice to meet you again. What would you like to talk about today?"

In [16]:
chain_with_history.invoke({"question": "Do you know what is my name"}, config)

'Yes, your name is Abby! How can I assist you today?'

In [17]:
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation2"}}

# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "Do you know what is my name"}, config)

'I don’t know your name unless you tell me. How can I assist you today?'